In [1]:
%pip install transformers
%pip install torch
%pip install datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [2]:
from datasets import load_dataset

ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")


c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load the tokenizer and classification model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
# 3 for yes/no/maybe classification
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1", num_labels=3)  

# Load dataset
ds = load_dataset("qiaojin/PubMedQA", "pqa_labeled")

# Define label mapping function for 'final_decision' field yes, no, maybe to 0, 1, 2
def label_mapping(final_decision):
    if final_decision == "yes":
        return 0
    elif final_decision == "no":
        return 1
    else:
        return 2

# Preprocess the dataset
def preprocess_function(examples):
    inputs = examples['question']
    # Grab answers from 'final_decision' field for true_answers labels
    targets = [label_mapping(ans) for ans in examples['final_decision']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = targets
    return model_inputs

# Select 5 examples for faster execution
small_ds = ds['train'].select(range(5))

# Tokenize the subset dataset
tokenized_ds = small_ds.map(preprocess_function, batched=True)

# Split the 5 examples: 4 for training and 1 for evaluation
train_test_split = tokenized_ds.train_test_split(test_size=1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate(eval_dataset=eval_dataset)
print("Evaluation Metrics:", metrics)
num_correct = 0

# Function for prediction using model and return the prediction as "yes", "no", or "maybe"
def predict_yes_no_maybe(question):
    # Use model prediction
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

    # Map numeric labels to "yes", "no", or "maybe"
    label_map = {0: "yes", 1: "no", 2: "maybe"}
    return label_map[predicted_label]

# Example predictions for 5 examples
for idx, example in enumerate(small_ds):
    question = example['question']
    # Extract true answer from 'final_decision' column
    true_answer = example['final_decision'] 
    # Use model to predict the answer
    predicted_label = predict_yes_no_maybe(question)
    
    print(f"Question: {question}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_label}")
    print("="*80)

    if true_answer == predicted_label:
        num_correct += 1

# Calculate accuracy
accuracy = num_correct / len(small_ds)
print(f"Accuracy: {accuracy}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                              
 33%|███▎      | 4/12 [00:07<00:13,  1.68s/it]

{'eval_loss': 1.9226144552230835, 'eval_runtime': 0.2935, 'eval_samples_per_second': 3.407, 'eval_steps_per_second': 3.407, 'epoch': 1.0}


                                              
 67%|██████▋   | 8/12 [00:14<00:06,  1.66s/it]

{'eval_loss': 1.8888282775878906, 'eval_runtime': 0.2696, 'eval_samples_per_second': 3.71, 'eval_steps_per_second': 3.71, 'epoch': 2.0}


                                               
100%|██████████| 12/12 [00:21<00:00,  1.83s/it]


{'eval_loss': 1.8782527446746826, 'eval_runtime': 0.315, 'eval_samples_per_second': 3.175, 'eval_steps_per_second': 3.175, 'epoch': 3.0}
{'train_runtime': 21.9826, 'train_samples_per_second': 0.546, 'train_steps_per_second': 0.546, 'train_loss': 1.0487988789876301, 'epoch': 3.0}


100%|██████████| 1/1 [00:00<?, ?it/s]


Evaluation Metrics: {'eval_loss': 1.8782527446746826, 'eval_runtime': 0.317, 'eval_samples_per_second': 3.155, 'eval_steps_per_second': 3.155, 'epoch': 3.0}
Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
True Answer: yes
Predicted Answer: yes
Question: Landolt C and snellen e acuity: differences in strabismus amblyopia?
True Answer: no
Predicted Answer: no
Question: Syncope during bathing in infants, a pediatric form of water-induced urticaria?
True Answer: yes
Predicted Answer: yes
Question: Are the long-term results of the transanal pull-through equal to those of the transabdominal pull-through?
True Answer: no
Predicted Answer: yes
Question: Can tailored interventions increase mammography use among HMO women?
True Answer: yes
Predicted Answer: yes
